In [1]:
import numpy as np 
import pandas as pd 

from sklearn.feature_extraction.text import CountVectorizer
from keras.layers import Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras import optimizers
import tensorflow as tf

import keras.backend as K

Using TensorFlow backend.


In [2]:
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

In [3]:
from utils import *
df = functions.parse_file(r"raw_data/EmoContext/train.txt", "EmoContext")
df.head(20)

,id,turn1,turn2,turn3,label
0,0,Don't worry I'm girl,hmm how do I know if you are,What's ur name?,others
1,1,When did I?,saw many times i think -_-,No. I never saw you,angry
2,2,By,by Google Chrome,Where you live,others
3,3,U r ridiculous,I might be ridiculous but I am telling the truth.,U little disgusting whore,angry
4,4,Just for time pass,wt do u do 4 a living then,Maybe,others
5,5,I'm a dog person,youre so rude,Whaaaat why,others
6,6,So whatsup,Nothing much. Sitting sipping and watching TV....,What are you watching on tv?,others
7,7,Ok,ok im back!!,"So, how are u",others
8,8,Really?,really really really really really,Y saying so many times...i can hear you,others
9,9,Bay,in the bay,😘 love you,others


In [4]:
text_data = []
for idx,row in df.iterrows():
    text_data.append("{}. {}. {}.".format(row['turn1'], row['turn2'], row['turn3']))

In [5]:
NR_WORDS = 5000
tokenizer = Tokenizer(num_words=NR_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                   lower=True,split=' ')

tokenizer.fit_on_texts(text_data)

X = tokenizer.texts_to_sequences(text_data)
X = pad_sequences(X)

In [6]:
words = {"others": 0, "angry": 1, "sad":2, "happy": 3}

def one_hot_vector(word):
    y = [0,0,0,0]
    y[words[word]] = 1
    return y

Y = []

for idx,row in df.iterrows():
    Y.append(one_hot_vector(row['label']))

Y = np.array(Y)

In [7]:
embed_dim = 256
lstm_out = 128
batch_size = 64

model = Sequential()
model.add(Embedding(NR_WORDS, embed_dim,input_length = X.shape[1]))
model.add(Dropout(0.3))
model.add(LSTM(lstm_out, dropout=0.5))
model.add(Dense(100))
model.add(Dense(4,activation='softmax'))
adam = optimizers.Adam(lr=0.01)
rmsprop = optimizers.RMSprop(lr=0.01, rho=0.9, epsilon=None, decay=0.0)
model.compile(loss = 'categorical_crossentropy', optimizer=rmsprop, metrics = ['accuracy', f1])


In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 154, 256)          1280000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 154, 256)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               197120    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               12900     
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 404       
Total params: 1,490,424
Trainable params: 1,490,424
Non-trainable params: 0
_________________________________________________________________


In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0, random_state = 42)

from sklearn.model_selection import KFold
# prepare cross validation
kfold = KFold(n_splits=6)
Y_train = np.array(Y_train)
# enumerate splits

total = len(train)
for train, validation in kfold.split(X_train):
    history = model.fit(X_train[train], Y_train[train],
                    validation_data=(X_train[validation], Y_train[validation]),
                    epochs=2, verbose=1, batch_size=batch_size,
                    class_weight={
                        0: total / len(np.where(Y_train[train][:,0]==1.0)[0]),
                        1: total / len(np.where(Y_train[train][:,1]==1.0)[0]),
                        2: total / len(np.where(Y_train[train][:,2]==1.0)[0]),
                        3: total / len(np.where(Y_train[train][:,3]==1.0)[0]),
})

Train on 25133 samples, validate on 5027 samples
Epoch 1/2
25133/25133 [==============================] - 307s 12ms/step - loss: 2.6069 - acc: 0.7539 - f1: 0.7182 - val_loss: 0.4638 - val_acc: 0.8518 - val_f1: 0.8393
Epoch 2/2
 2944/25133 [==>...........................] - ETA: 3:42 - loss: 1.5938 - acc: 0.8607 - f1: 0.8539

KeyboardInterrupt: 

In [ ]:
model_json = model.to_json()
with open("lstm_normal_model.json", "w") as outfile:
    outfile.write(model_json)
model.save_weights("lstm_normal_12_epochs_cv.h5")

In [ ]:
df_test = functions.parse_file(r"raw_data/EmoContext/devwithoutlabels.txt", "EmoContext")
df_test.head()

In [ ]:
text_data = []

for idx,row in df_test.iterrows():
    text_data.append("{}. {}. {}.".format(row['turn1'], row['turn2'], row['turn3']))

res = model.predict(X_test, batch_size=128, verbose=1)
res[3]

In [ ]:
revers_words = {0:"others", 1:"angry", 2:"sad", 3:"happy"}

def softmax_convert(res):
    max_i = 0
    max_v = 0
    for i in range(0,4):
        if res[i] > max_v:
            max_v = res[i]
            max_i = i
    return revers_words[max_i]

In [ ]:
results = []
for r in res:
    results.append(softmax_convert(r))
    
df_test['label'] = results
df_test.head(30)
df_test.to_csv("lstm_normal_12_epochs_cv.txt",index=False , sep="\t")